In [15]:
from googleapiclient.discovery import build

api_key = input()

    
youtube = build('youtube','v3',developerKey = api_key)

[YT API KEY]

In [16]:
ytid = input()




YT VIDEO ID

In [22]:
dictlist = []

dict_output = youtube.commentThreads().list(part="snippet,replies", videoId=ytid, maxResults = 100).execute()
dictlist.append(dict_output)
npt = dict_output.get('nextPageToken')

while npt:
    dict_output = youtube.commentThreads().list(part="snippet,replies", videoId=ytid, pageToken = npt, maxResults = 100).execute()
    dictlist.append(dict_output)
    npt = dict_output.get('nextPageToken')

comments = []  
for dict in dictlist:
    for thread in dict['items']:
        comment = []
        commenttext = thread['snippet']['topLevelComment']['snippet']['textDisplay']
        replycount = thread['snippet']['totalReplyCount']

        if replycount>0:
            for replycomment in thread['replies']['comments']:
                replycommenttext = replycomment['snippet']['textDisplay']
                comment.append(replycommenttext)
            comment.insert(0,commenttext)
        else:
            comment.append(commenttext)  

        comments.append(comment)
        
    

In [23]:
comments

[['This trailer is 1080p...'],
 ['「ところで、また攻殻を観に行くの？DVD、Blu-rayで観てもまだ足りない？」<br>「俺は攻殻が好きなの。」'],
 ['Hell yes'],
 ['この考えと技術力を当時実行したことは本当に凄いことだと思う<br>ただこの映画の魅力(？)の一つとされている謎の言語音楽は正直うるさいだけで自分は嫌い<br>イノセンスにも多用されていてバトーxトグサが観たいのに音楽が邪魔してる'],
 ['人間が人間であるための部品が決して少なくないように、自分が自分であるためには驚くほど多くのものが必要なの。 <br>他人を隔てるための顔、それと意識しない声、目覚めのときに見つめる手、幼かった頃の記憶、未来の予感。それだけじゃないわ、私の電脳がアクセスできる膨大な情報やネットの広がり。それらすべてが私の一部であり、私という意識そのものを生み出し、そして同時に…私をある限界に制約し続ける…！'],
 ['はっきり言って、アニメの分野では日本はもう韓国に負けている。韓国の方が優れている。日本人は負けを認めたくないだけ。'],
 ['4K Trailer<br>Video is 1080p smh'],
 ['Way better than the Hollywood remake. There, I said it!'],
 ['The video is only 1080p LMAO'],
 ['I hope they <i>Enhance</i> the <b>3D</b>'],
 ['このホロホロとした質感良いな'],
 ['Avalonも、もう一度劇場で見てみたい。ビューティフル・ドリーマーもリマスター、いやリメイクしてほしい。'],
 ['cool season 2 of 2045 must be close'],
 ['ほんとだー！\u3000画質がクリアで良くなってる気がする'],
 ['The way these anime is drawn I&#39;m in awe. I love the old way of drawings'],
 ['pogeroni i say'],
 ['How can we get IMAX English Sub exhibitions in Brazil?'